In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config('spark.shuffle.useOldFetchProtocol', 'true'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
customers_df = spark.read \
.format("csv") \
.option("inferSchema", True) \
.load("/public/trendytech/retail_db/customers/part-00000")

In [3]:
customers_final_df = customers_df.toDF("customer_id", "customer_fname", "customer_lname", "customer_email", "customer_password", "customer_street", "customer_city", "customer_state", "customer_zipcode")

In [4]:
customers_final_df.show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|             725|
|          4|          Mary|         Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|            CA|          

In [5]:
spark.sql("create database itv009538_bucketingdb")

""


In [6]:
customers_final_df.write \
.format("parquet") \
.mode("overwrite") \
.bucketBy(4, "customer_id") \
.saveAsTable("itv009538_bucketingdb.customersnew")

In [8]:
spark.sql("select count(*) from itv009538_bucketingdb.customersnew where customer_state = 'TX'")

count(1)
635


In [10]:
spark.sql("select * from itv009538_bucketingdb.customersnew where customer_id = 10")

customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
10,Melissa,Smith,XXXXXXXXX,XXXXXXXXX,8598 Harvest Beac...,Stafford,VA,22554
